# Scraping roguelike articles on the web

This Python notebook describes the data collection process for roguelike universe. If you have not yet installed the requirements, you can do it by running:

`pip install -r requirements.txt`

## Sourcing game titles

We prepared a [list of roguelike games](https://en.wikipedia.org/wiki/List_of_roguelikes) from Wikipedia as a starting point. For indicators of out-of-genre influences, we sourced 10,000+ video game titles from Pastebin as uploaded by the user ________. 

First, we setup read/write functions that are friendly with international unicode characters, because the web may contain all kinds of character code points.

In [168]:
import os
import io
import json
import pandas as pd

def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message('Loaded {}'.format(path)))
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message('Written to {}'.format(path)))
    
def __success(text):
    return '  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return '!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return '??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return '   |MSG| {}'.format(text).encode('utf-8')

Here is a sample of the list of roguelike games from Wikipedia:

In [169]:
roguelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelikes.csv'), skip_blank_lines=True)
roguelikes.head()

,Name,RogueTemple,Link,Released,Updated,Developer,Theme,Influences
0,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,2010/05/06,2010/05/06,Dinofarm Games,Fantasy,Rogue
1,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr,2014/02/20,2015/02/07,Ratz 'N' Godz,Fantasy,"Dungeon Crawl Stone Soup, Dominions 4: Thrones..."
2,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,2005/00/00,2005/06/11,Phr00t,"Science Fiction, Alien Planets, Futuristic",NetHack
3,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,2009/07/06,2009/10/06,Phr00t,"Science Fiction, Alien Planets, Futuristic",3059
4,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,2011/10/25,2015/02/13,Phr00t,"Science Fiction, Alien Planets, Futuristic","3059, 3069, Fallout, Minecraft"


In addition, roguelike-like games:

In [170]:
roguelikelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelike-likes.csv'), skip_blank_lines=True)
roguelikelikes.head()

,Name,Released,Updated,Developer,Theme,Influences
0,ToeJam & Earl,1991,NaN,Johnson Voorsanger Productions,Fantasy,NaN
1,Diablo,1996,NaN,Blizzard North,Fantasy,NaN
2,Diablo II,2000,NaN,Blizzard Entertainment,Fantasy,NaN
3,Lost Labyrinth,2001,2011.0,Lost Labyrinth,Fantasy,NaN
4,Strange Adventures In Infinite Space,2002,2004.0,"Rich Carlson, Iikka Keränen",Space science fiction,NaN


And a sample of the list of video games:

In [171]:
video_games = pd.read_json(os.path.join(os.getcwd(), 'games.json'))
video_games.head(10)

,title,year
0,$hop-n-$pree,2009
1,'43 - One Year After,1986
2,'89 Denno Kyusei Uranai,1988
3,'Nam 1965-1975,1991
4,'Splosion Man,2009
5,'Til Death Do Us Part,2013
6,(Almost) Total Mayhem,2011
7,(Not) Just another Space Shooter,2004
8,(T)Raumschiff Surprise - Periode 1,2004
9,*NSYNC Hotline Phone and Fantasy CD-Rom Game,2001


## Building a corpus 

Before we can do any text analysis, we need to build a corpus in which to operate on.

### 1. RogueTemple

RogueTemple Wiki collects a detailed description of roguelike games.

In [172]:
import requests

def scrape_mediawiki_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    response = requests.get(url, headers=headers, timeout=(9.1, 12.1))
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    content = [node.text.strip() for node in soup.select('#mw-content-text') if node.text]
    return ''.join(content)

In [173]:
# Sample scrape
content = scrape_mediawiki_url('http://roguebasin.roguelikedevelopment.org/index.php?title=100_Rogues')
print(content)

100 Rogues



Stable game



Developer

Dinofarm Games



Theme

Fantasy



Influences

Rogue



Released

 (?)



Updated

May 6, 2010 (?)



Licensing

Commercial



P. Language





Platforms

iPhone



Interface

Graphical Tiles



Game Length

Medium



Official site of 100 Rogues




100 Rogues is an original Roguelike for the iPhone, iPod Touch and iPad devices.  It features two playable classes with unique skill trees (similar to Diablo), several tilesets, original SNES-style music, and fully animated pixel art.  100 Rogues was developed from scratch for the iPhone OS devices, and has a click to move control scheme rather than a virtual d-pad.

Contents

1 Gameplay
2 Combat
3 Skills
4 Reception


 Gameplay
A game of 100 Rogues is a linear progression through 12 dungeon levels spread across 3 worlds: The Bandit Hole, The Dungeon, and Hell. The first three levels of each world consist of randomly-generated maps initially populated by Mobs of different Monsters. After these Mobs a

### 2. Wikipedia

Searching on Wikipedia can be done with two dozens line of code.

In [174]:
import wikipedia

def scrape_wiki_id(pageid):
    page = wikipedia.page(pageid=pageid)    
    print_wiki_page(page)
    
def scrape_wiki(title):
    try:
        searchstring = title
        page = wikipedia.page(searchstring, auto_suggest=False)
#         print_wiki_page(page)
        return page
    except wikipedia.DisambiguationError:
        try:
            searchstring = '{} (video game)'.format(title).replace(' ', '_')
            page = wikipedia.page(searchstring, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except wikipedia.DisambiguationError:
            try:
                searchstring = '{} (Unix video game)'.format(title).replace(' ', '_')
                page = wikipedia.page(searchstring, auto_suggest=False)
    #             print_wiki_page(page)
                return page
            except:
                print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
        except:
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            page = wikipedia.page(title, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except:
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    
def print_wiki_page(page):
    print(page.title)
    print(page.content)
    print(page.references)  

In [175]:
# Test Wikipedia crawl
print(scrape_wiki('Rogue Legacy'))

<WikipediaPage 'Rogue Legacy'>


### 3. DuckDuckGo

We also source a list of potential interesting webpages via an internet search engine, DuckDuckGo.

In [176]:
import bs4
import time
import requests
import urllib.parse

def scrape_duckduckgo(keywords, developer=""):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    searchstring = u'"{}" AND {} AND game AND (mortem OR history OR developer OR review)'.format(keywords, developer)
    q = u'http://duckduckgo.com/html/?q={}'.format(urllib.parse.quote(searchstring.encode('utf-8')))
    print(q)
                                                   
    response = requests.get(q, headers=headers, timeout=(9.1, 12.1))
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    links = []
    links = [node.get('href') for node in soup.select('a.result__a')]
    return links

In [177]:
scrape_duckduckgo('Ancient Domains of Mystery', 'Thomas Biskup')

http://duckduckgo.com/html/?q=%22Ancient%20Domains%20of%20Mystery%22%20AND%20Thomas%20Biskup%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29


['https://store.steampowered.com/video/333300',
 'https://forum.zoneofgames.ru/topic/36951-adom-ancient-domains-of-mystery/',
 'https://en.wikipedia.org/wiki/Ancient_Domains_of_Mystery',
 'https://www.cultureofgaming.com/ancient-domains-of-mystery-adom-review/',
 'https://www.youtube.com/watch?v=ChtBuBrFYc8',
 'https://lgdb.org/game/adom-ancient-domains-of-mystery',
 'https://www.turkaramamotoru.com/en/ancient-domains-of-mystery-181815.html',
 'https://stillnessinthestorm.com/2018/08/the-mystery-of-ancient-nuclear-war/',
 'https://www.giantbomb.com/ancient-domains-of-mystery/3030-6149/',
 'https://wiki2.org/en/Ancient_Domains_of_Mystery',
 'http://RuTracker.org/forum/viewtopic.php?t=5451914',
 'https://ancient-domains-of-mystery.ru.uptodown.com/',
 'https://gamesmojo.com/games/adventure/adom-ancient-domains-of-mystery',
 'https://steamcommunity.com/sharedfiles/filedetails/?id=258925365',
 'http://gaming.wikia.com/wiki/Ancient_Domains_of_Mystery',
 'https://yepdownload.com/ancient-domai

## Scrape the corpus

With the functions above we can collect a corpus.

In [178]:
def scrape(url, title):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    try:
        print(__message('Scraping {} ...'.format(url)))
        response = requests.get(url, headers=headers, timeout=(9.1, 12.1))
    except Exception as e:
        print(__failure('Failed to load {}'.format(url)))
        print(e)
        return None
    
    html = response.text
    if html and any(word in html.lower() for word in ['tutorial']):
        return None
    if html and any(word in html.lower() for word in [title, 'rogue', 'procedural', 'generation', 'mortem', 'review', 'history', 'develop', 'idea', 'inspir']):
        print(__message('Found article'))
        soup = bs4.BeautifulSoup(response.text, 'lxml')
        selections = soup.select('body > p') + soup.select('div > p') + soup.select('table td')
        content = [node.text.strip() for node in selections]
        return ''.join(content)
    return None

In [179]:
# For Roguelike games, we build a corpus with RogueTemple and DuckDuckGo
corpus = []
    
for index, roguelike in roguelikes.iterrows():
    print(roguelike)
    title = roguelike['Name']
    if not isinstance(title, str):
        continue
    text = []
    
    rogue_temple = scrape_mediawiki_url(roguelike['RogueTemple'])
    text.append(rogue_temple)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:25]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link, title)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus.json', corpus)

Name                                                  100 Rogues
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                   http://www.100rogues.com/
Released                                              2010/05/06
Updated                                               2010/05/06
Developer                                         Dinofarm Games
Theme                                                    Fantasy
Influences                                                 Rogue
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22100%20Rogues%22%20AND%20Dinofarm%20Games%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://100rogues.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.dinofarmgames.com/forum/index.php?threads/100-rogues-on-android.1501/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.dinofarmgames.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scrap

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.ag.ru/games/post-mortem/review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://futureofwork.nobl.io/future-of-work/definitive-guide-to-agile-retrospectives-and-post-mortem-meetings ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.ign.com/articles/2003/02/25/post-mortem-review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/586370 ...'
b'   |MSG| Scraping http://www.y8.com/games/post_mortem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://developer.ibm.com/javasdk/tools/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://voidinteractive.net/ ...'
b'   |MSG| Found article'
Name                                                        3069
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                      http://sites.google.com/site/free3069/
Released                                              2009/07/06
Updated                   

b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=P9JYgBmfL-Y ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamedevelopment.tutsplus.com/articles/15-analyses-post-mortems-and-game-design-docs--gamedev-11554 ...'
b'   |MSG| Scraping https://www.gamesparks.com/blog/gdc-and-game-connection-post-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://adventuregamers.com/articles/view/17557 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/video/263360?l=czech ...'
b'   |MSG| Scraping http://gamestorming.com/pre-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://small-games.info/?go=game&c=4&i=12408 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.gameboomers.com/wtcheats/pcPp/Post_Mortem.htm ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamasutra.com/features/postmortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://3089game.wordpress.com/blog/ ...'
b'  

b'   |MSG| Found article'
Name                                              Advanced Rogue
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                http://rogue.rogueforge.net/
Released                                              2005/00/00
Updated                                               2007/08/01
Developer      Michael Morgan, Ken Dalka, Roguelike Restorati...
Theme                                                        NaN
Influences                                    Rogue, Super-Rogue
Name: 8, dtype: object
http://duckduckgo.com/html/?q=%22Advanced%20Rogue%22%20AND%20Michael%20Morgan%20OR%20%20Ken%20Dalka%20OR%20%20Roguelike%20Restoration%20Project%20OR%20%20others%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://gaming.stackexchange.com/q/246887 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.roguelikedevelopment.org/archive/ ...'
b'   |MSG| Found article'
b

b'   |MSG| Found article'
b'   |MSG| Scraping https://www.inverse.com/article/15744-rogue-one-characters-ships-and-more-details-revealed-in-visual-story-guide-preview ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.zophar.net/pdroms/gba/agb_rogue.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.thisisbarry.com/single-post/2016/10/26/Donnie-Darko-2001-Full-Plot-and-Ending-Explained ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://superherojacked.com/2017/08/01/donnie-yen-workout/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.johnyorkestory.com/2017/09/the-essential-dos-and-donts-of-writing-a-video-game/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://student.unsw.edu.au/interview-dos-and-donts ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.rookandrogue.com/about-us.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.michaelpage.com.au/advice/career-advice/changing-jobs/top-do-s-and-don

...

b'   |MSG| Found article'
b'   |MSG| Scraping https://www.moddb.com/games/mortem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/MortemManor/?ref=py_c ...'
b'   |MSG| Scraping https://manishearth.github.io/blog/2015/05/28/github-streak-end-game-and-post-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://wiki2.org/en/Postgame_Mortem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/586370 ...'
b'   |MSG| Scraping https://gamasutra.com/features/postmortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://empiresatwarblog.blogspot.com/2017/10/mortem-et-gloriam-crusades-game.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.ign.com/articles/2003/02/25/post-mortem-review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://en.wiktionary.org/wiki/post_mortem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.indiedb.com/games/nox-mortem ...'
b'   |MSG| Found article'
N

http://duckduckgo.com/html/?q=%22XLarn%22%20AND%20Swinfjord-Games%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://swinfjord-games.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamesmojo.com/games/indie/xlarn ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/post-mortem-review/1900-2911836/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/360030/XLarn/ ...'
b'   |MSG| Scraping https://www.moddb.com/games/xlarn ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.indiedb.com/company/swinfjord-games ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamemonitoring.net/ru/games/xlarn ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://AppAgg.com/developer/swinfjord-games/?hl=ru ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.qwant.com/game/xlarn?l=pl ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.felicedunas.com/male-and-female-differences-and-strengths-the-yin-yang-perspective/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://buddhaweekly.com/guan-yin-ten-great-protections-goddess-mercy-avalokiteshvara-bodhisattva-compassion/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.davidrevoy.com/article43/yin-and-yang-of-world-hunger ...'
Name                                                     Yodanji
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link           http://www.kemco.jp/sp/games/yodanji/en/index....
Released                                              2017/00/00
Updated                                               2017/06/05
Developer                                                  Kemco
Theme                                                      Yokai
Influences                                                   NaN
Name: 255, dtype: object
http://duckduckgo.com/html/?q=%2

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.gameboomers.com/wtcheats/pcPp/Post_Mortem.htm ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamejolt.com/games/mortem/223601 ...'
b'   |MSG| Scraping https://gamasutra.com/blogs/KengJin/20151022/255822/PostMortem_Making_a_Fan_Game__Why_How_and_What_happens_next.php ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.logigear.com/magazine/test-methods-and-metrics/same-or-different-retrospectives-and-post-mortems/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://manishearth.github.io/blog/2015/05/28/github-streak-end-game-and-post-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://nighthoodgames.itch.io/mortem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.atlasobscura.com/articles/victorian-post-mortem-photographs ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamepressure.com/download.asp?ID=1993 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping htt

http://duckduckgo.com/html/?q=%22Zone%22%20AND%20Quarry%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.gamespot.com/reviews/post-mortem-review/1900-2911836/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.oldpcgaming.net/post-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.thewargameszone.org/60/The-Zones ...'
b'   |MSG| Scraping https://adventuregamers.com/articles/view/17557 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://theisozone.com/downloads/pc/windows-games/post-mortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamasutra.com/features/postmortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.zoneofgames.ru/games/637/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.torrent-zone.com/mortem-postmortem/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=P9JYgBmfL-Y ...'
b'   |MSG| Found article

In [180]:
# For Roguelike-like games, we build a corpus with Wikipedia and DuckDuckGo
# corpus = read_json('corpus-roguelike-like.json')
# if not corpus:
corpus = []
    
for index, roguelike in roguelikelikes.iterrows():
    print(roguelike)
    title = roguelike['Name']
    text = []
    
    page = scrape_wiki(title)
    if page:
        text.append(page.content)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:20]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link, title)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus-roguelike-like.json', corpus)

Name                           ToeJam & Earl
Released                                1991
Updated                                  NaN
Developer     Johnson Voorsanger Productions
Theme                                Fantasy
Influences                               NaN
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22ToeJam%20%26%20Earl%22%20AND%20Johnson%20Voorsanger%20Productions%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://gaming.wikia.com/wiki/ToeJam_%26_Earl_Productions ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://lastdaydeaf.com/toejam-earldev-johnson-voorsanger-productions-sega-genesis-1991/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://toejamandearl.wikia.com/wiki/ToeJam_%26_Earl_in_Panic_on_Funkotron ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://segaretro.org/ToeJam_%26_Earl ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/pages/ToeJam-Earl-Prod

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://duckduckgo.com/html/?q=%22Diablo%22%20AND%20Blizzard%20North%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.youtube.com/watch?v=VscdPA6sUkc ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://us.battle.net/forums/en/d3/topic/7415795753 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://idclips.com/rev/blizzard+diablo/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.pcgamer.com/diablo-designer-david-breviks-full-gdc-post-mortem-is-now-online/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diablowiki.net/Blizzard_North ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diabloii.net/blog/comments/diablo-3-post-mortem-jay-wilson-pt1 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://ltclip.com/rev/diablo+blizzard/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamasutra.com/view/feature/131533/postmortem_blizzards_diablo_ii.php ...'
b'   |MSG

b'   |MSG| Found article'
b'   |MSG| Scraping http://www.g4g.it/2009/10/01/strange-adventures-in-infinite-space/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamezebo.com/2010/11/18/strange-adventures-infinite-space-review/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/strange-adventures-in-infinite-space-review/1900-2856375/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.myabandonware.com/game/strange-adventures-in-infinite-space-3qy ...'
b'   |MSG| Scraping http://igrotop.com/games/strange_adventures_in_infinite_space/info ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.game-ost.ru/games/6212/strange_adventures_in_infinite_space/meta/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://wiki2.org/en/Strange_Adventures_in_Infinite_Space ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://vikimy.com/l-en/Strange_Adventures_In_Infinite_Space ...'
b'   |MSG| Found article'
b'   |MSG| Scrapin

b'   |MSG| Found article'
b'   |MSG| Scraping http://bestgamer.net/load/3125-the-binding-of-isaac-edmund-mcmillen-eng-p.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://tcrf.net/The_Binding_of_Isaac ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/workshop/filedetails/?id=895960454 ...'
b'   |MSG| Found article'
Name          FTL: Faster Than Light
Released                        2012
Updated                          NaN
Developer               Subset Games
Theme          Space science fiction
Influences                       NaN
Name: 8, dtype: object
http://duckduckgo.com/html/?q=%22FTL%3A%20Faster%20Than%20Light%22%20AND%20Subset%20Games%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping http://gaming.wikia.com/wiki/FTL:_Faster_Than_Light ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://speed-new.com/ftl-faster-than-light-full-pc-game ...'
b'   |MSG| Found article'
b'   |MSG| Scrap

...

b'   |MSG| Found article'
Name          The Binding of Isaac: Rebirth
Released                               2014
Updated                                 NaN
Developer                           Nicalis
Theme                               Fantasy
Influences                              NaN
Name: 22, dtype: object
http://duckduckgo.com/html/?q=%22The%20Binding%20of%20Isaac%3A%20Rebirth%22%20AND%20Nicalis%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://gamesmojo.com/games/action/the-binding-of-isaac-rebirth ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://maddownload.com/games/rpg/the-binding-of-isaac-rebirth/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/250900/The_Binding_of_Isaac_Rebirth/ ...'
b'   |MSG| Scraping http://fragrun.com/games/action/the-binding-of-isaac-rebirth ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://playstationgamer.org/games/action/the-binding-of

b'   |MSG| Scraping https://wccftech.com/review/hand-of-fate-2/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://oceanofgames.com/hand-of-fate-free-download/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/hand-of-fate-2-review/1900-6416814/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gog.com/game/hand_of_fate_2_outlands_and_outsiders ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.gamerevolution.com/review/67003-hand-of-fate-review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://gamestorrent.co/hand-of-fate-2-outlands-and-outsiders-plaza.html ...'
b'   |MSG| Scraping https://www.pcpowerplay.com.au/review/hand-of-fate-2,477783 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://speed-new.com/hand-of-fate-full-pc-game ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://unity3d.com/showcase/case-stories/hand-of-fate ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.

b'??WARN?? Search term "Infinite Space III: Sea of Stars" returned nothing'
http://duckduckgo.com/html/?q=%22Infinite%20Space%20III%3A%20Sea%20of%20Stars%22%20AND%20Rich%20Carlson%20OR%20%20Iikka%20Ker%C3%A4nen%20AND%20game%20AND%20%28mortem%20OR%20history%20OR%20developer%20OR%20review%29
b'   |MSG| Scraping https://www.imdb.com/title/tt1323932/fullcredits ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/WeirdWorldsReturnToInfiniteSpace ...'
b'   |MSG| Scraping https://www.facebook.com/infinitespace3 ...'
b'   |MSG| Scraping http://doom.wikia.com/wiki/List_of_notable_WADs ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://doomwiki.org/wiki/Hacx ...'
b'   |MSG| Scraping https://doomwiki.org/wiki/Requiem ...'
b'   |MSG| Scraping https://store.steampowered.com/app/698100/Protagon_VR/ ...'
b'   |MSG| Scraping http://doom.wikia.com/wiki/Requiem ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://onemandoom.blogspot.com/p/index-of-reviews.html ...'


b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/app/265000/discussions/0/365163537817753926/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.steamgifts.com/discussion/VYsDF/something-to-read-interesting-history-of-indie-dev-betadwarf-makers-of-forced-and-forced-showdown ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.destructoid.com/review-forced-showdown-368410.phtml ...'
b'   |MSG| Scraping https://ru-ru.facebook.com/BetaDwarf/posts ...'
b'   |MSG| Scraping http://cogconnected.com/review/forced-showdown-review/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://linuxgamenews.com/post/63208838351/the-incredible-story-of-betadwarf-and-forced ...'
b'   |MSG| Scraping https://gamingbolt.com/forced-review ...'
b'   |MSG| Scraping http://www.grabthegames.com/review-forced-showdown.html ...'
b'   |MSG| Found article'
Name          Pixel Cave
Released            2016
Updated              NaN
Developer       Megabyte
Theme         

b'   |MSG| Scraping https://saveorquit.com/2017/10/07/review-heat-signature/ ...'
b'   |MSG| Scraping http://heatsig.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.rockpapershotgun.com/2017/09/26/heat-signature-review/ ...'
b'   |MSG| Scraping https://gamesmojo.com/games/action/heat-signature ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://heatsignature.wikia.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.pcgamesn.com/heat-signature/heat-signature-release-date-steam-trading-cards ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.polygon.com/2017/9/22/16350330/heat-signature-review ...'
b'   |MSG| Scraping https://waypoint.vice.com/en_us/article/evpvmk/watch-us-steal-cool-spaceships-in-heat-signature ...'
b'   |MSG| Found article'
Name                   FARA
Released               2018
Updated                 NaN
Developer     Brian Roberts
Theme               Fantasy
Influences              NaN
Name: 37, dtype: object
b'?